<a href="https://colab.research.google.com/github/tomheston/fragility-metrics/blob/main/notebooks/binary_2x2_fixed_margin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title
# Fragility Metrics Toolkit: Matched / Fixed-Margin Binary Outcomes (2×2)
# 19-NOV-2025
#
# Layout (generic 2×2 fixed-margin / matched design):
#      | Outcome A | Outcome B |
# ArmA |     a     |    b      |
# ArmB |     c     |    d      |
#
# Input:
#   a, b, c, d  (non-negative integers; margins fixed by design)
#
# Output (p–fr–nb):
#   p   : McNemar χ² baseline p-value (two-sided, uncorrected)
#   fr  : PFI (Percent Fragility Index, decimal scale) via McNemar path
#   nb  : RQ  (Risk Quotient, Neutrality Boundary Framework)
#
# Notes
# - This calculator is for matched / fixed-margin 2×2 designs.
# - Baseline significance test: McNemar χ² (two-sided, df = 1, no correction).
# - PFI is computed along a fixed-margins McNemar path:
#       (a, b, c, d) → (a + x, b - x, c - x, d + x)
#   with continuous x, using McNemar χ² on discordant pairs.
#   Normalization: |x| / (N/4) for [0,1] scale, orthogonal to RQ.
#
# IF YOU USE THIS CALCULATOR PLEASE CITE:
# Heston, T. F. (2025). Fragility Metrics Toolkit [Software]. Zenodo.
# https://doi.org/10.5281/zenodo.17254763
#
# © Thomas F. Heston 2025. CC-BY 4.0

# ----- SciPy availability guard -----
try:
    import scipy
except ImportError:
    try:
        import subprocess
        import sys
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "scipy"])
        import scipy
    except Exception:
        print("Please install scipy: pip install scipy")
        raise

import numpy as np
from scipy.stats import chi2

ALPHA = 0.05


# ---------- Core utilities ----------

def n_total(a, b, c, d):
    """Total sample size."""
    return a + b + c + d


def mcnemar_p(a, b, c, d):
    """
    McNemar χ² test (two-sided, uncorrected).

    For a matched / fixed-margin 2×2 design, the McNemar statistic is:
        χ² = (b - c)^2 / (b + c), df = 1  (no continuity correction).

    Returns the two-sided p-value from the χ²(1) distribution.
    """
    b_c = b + c
    if b_c == 0:
        # No discordant pairs -> no evidence against H0: p_b = p_c
        return 1.0
    chi2_stat = (b - c) ** 2 / b_c
    p = chi2.sf(chi2_stat, df=1)  # upper tail
    return float(p)


def p_along_x_mcnemar(a, b, c, d, x):
    """
    Fixed-margins McNemar path:

        (a, b, c, d) → (a + x, b - x, c - x, d + x)

    used to define the Percent Fragility Index (PFI) via a continuous
    perturbation x while keeping row/column margins fixed.
    """
    aa, bb, cc, dd = a + x, b - x, c - x, d + x
    if min(aa, bb, cc, dd) < 0:
        return np.nan
    # McNemar on discordant: bb vs cc
    return mcnemar_p(aa, bb, cc, dd)


# ---------- PFI + RQ (fixed-margin, McNemar path) ----------

def compute_pfi_and_rq(a, b, c, d, alpha=ALPHA, tol=1e-10, max_iter=200):
    """
    Compute:
      - PFI (Percent Fragility Index, decimal scale) via McNemar path
      - RQ  (Risk Quotient, from |ad - bc| geometry)

    Returns:
      {
        "PFI": PFI_decimal_or_None,
        "RQ":  rq_or_None
      }
    """
    a, b, c, d = map(float, (a, b, c, d))
    n = a + b + c + d
    if n <= 0 or min(a, b, c, d) < 0:
        return {"PFI": None, "RQ": None}

    # p-independent robustness geometry (unchanged):
    # RRI = |ad - bc| / n
    # RQ  = 4 * RRI / n
    rri = abs(a * d - b * c) / n
    rq = (4.0 * rri) / n

    # Feasible x along fixed-margins path
    x_min = -min(a, d)
    x_max = min(b, c)

    # Baseline McNemar p-value
    p0 = mcnemar_p(a, b, c, d)
    if not np.isfinite(p0):
        return {"PFI": None, "RQ": rq}

    sig0 = (p0 <= alpha)

    def f(x):
        """Root function: p(x) - alpha."""
        return p_along_x_mcnemar(a, b, c, d, x) - alpha

    # Search interval: toward/away from boundary
    # McNemar boundary at b = c, i.e., x = (b - c)/2
    x_boundary = (b - c) / 2.0

    if sig0:
        # Significant: move toward boundary to lose sig
        lo, hi = (0.0, x_boundary) if x_boundary >= 0 else (x_boundary, 0.0)
    else:
        # Nonsig: move away from boundary to gain sig
        if x_boundary >= 0:
            lo, hi = (0.0, x_min)
        else:
            lo, hi = (0.0, x_max)

    # Clamp to feasible
    if lo > hi:
        lo, hi = hi, lo
    lo = max(lo, x_min)
    hi = min(hi, x_max)

    x_star = None

    if lo != hi:
        flo, fhi = f(lo), f(hi)
        # Grid search if no bracket
        if not (np.isfinite(flo) and np.isfinite(fhi)) or flo * fhi > 0:
            GRID = 400
            xs = np.linspace(lo, hi, GRID + 1)
            fs = np.array([f(x) for x in xs])
            found = False
            for i in range(GRID):
                if (np.isfinite(fs[i]) and np.isfinite(fs[i+1]) and
                        fs[i] * fs[i+1] <= 0):
                    lo, hi = xs[i], xs[i+1]
                    flo, fhi = fs[i], fs[i+1]
                    found = True
                    break
            if not found:
                x_star = None

        # Bisection
        if lo != hi and x_star is None:
            for _ in range(max_iter):
                mid = 0.5 * (lo + hi)
                fmid = f(mid)
                if not np.isfinite(fmid):
                    mid = np.nextafter(mid, hi)
                    fmid = f(mid)
                    if not np.isfinite(fmid):
                        break
                if abs(hi - lo) < tol:
                    x_star = mid
                    break
                if flo == 0:
                    x_star = lo
                    break
                if fhi == 0:
                    x_star = hi
                    break
                if flo * fmid <= 0:
                    hi, fhi = mid, fmid
                else:
                    lo, flo = mid, fmid
            else:
                x_star = 0.5 * (lo + hi)

    PFI_dec = None
    if x_star is not None:
        UFI_cont = abs(x_star)
        PFI_dec = (4.0 * UFI_cont) / n  # N/4 normalization for [0,1]

    return {"PFI": PFI_dec, "RQ": rq}


# ---------- High-level calculator: binary 2×2 fixed-margin ----------

def calculate_binary_2x2_fixed_margin(a, b, c, d, alpha=ALPHA):
    """
    High-level calculator for matched / fixed-margin 2×2 binary outcomes.

    Parameters
    ----------
    a, b, c, d : int
        Cell counts in the 2×2 table with fixed margins.

    Returns
    -------
    result : dict
        {
          "p":  <McNemar two-sided p-value>,
          "fr": {
             "PFI": <Percent Fragility Index, decimal> or None
          },
          "nb": {
             "RQ":  <Risk Quotient> or None
          }
        }
    """
    # Baseline p-value (McNemar χ²)
    p_val = mcnemar_p(a, b, c, d)

    # Fragility + robustness from McNemar path
    pfi_rq = compute_pfi_and_rq(a, b, c, d, alpha=alpha)

    result = {
        "p": p_val,
        "fr": {
            "PFI": pfi_rq.get("PFI")
        },
        "nb": {
            "RQ": pfi_rq.get("RQ")
        }
    }
    return result


# ---------- CLI: minimal p–fr–nb output ----------

def main():
    print("Enter 2×2 fixed-margin table cells as integers.")
    a = int(input("a (Arm A outcome A): ").strip())
    b = int(input("b (Arm A outcome B): ").strip())
    c = int(input("c (Arm B outcome A): ").strip())
    d = int(input("d (Arm B outcome B): ").strip())

    res = calculate_binary_2x2_fixed_margin(a, b, c, d, alpha=ALPHA)

    print("\n================ p–fr–nb =================")
    print(f"p  = {res['p']:.6f}  (McNemar χ², two-sided)")

    fr = res["fr"]
    print("fr:")
    if fr["PFI"] is not None:
        print(f"  PFI = {fr['PFI']:.6f}")
    else:
        print("  PFI = None")

    nb = res["nb"]
    print("nb:")
    if nb["RQ"] is not None:
        print(f"  RQ  = {nb['RQ']:.6f}")
    else:
        print("  RQ  = None")
    print("==========================================")

if __name__ == "__main__":
    main()

Enter 2×2 fixed-margin table cells as integers.
a (Arm A outcome A): 5
b (Arm A outcome B): 18
c (Arm B outcome A): 14
d (Arm B outcome B): 28

================ p–fr–nb =================
p  = 0.479500  (McNemar χ², two-sided)
fr:
  PFI = 0.105444
nb:
  RQ  = 0.106036
